In [130]:
import pandas as pd
import numpy as np
import torch
from importlib import reload
import CIR_
from CIR_ import CIR
reload(CIR_)
# import CIR2_
# from CIR2_ import CIR2
# reload(CIR2_)

import sys
import numpy
# numpy.set_printoptions(suppress=True, threshold=sys.maxsize)

X = pd.read_csv('foregroundX.csv')
Y = pd.read_csv('foregroundY.csv')
Xt = pd.read_csv('backgroundX.csv')
Yt = pd.read_csv('backgroundY.csv')

X = X.iloc[:, 1:]
Xt = Xt.iloc[:, 1:]
Y = Y.iloc[:, 1:]
Yt = Yt.iloc[:, 1:]

O = pd.read_csv('output.csv')
O = pd.DataFrame(O)
O = O.iloc[:, 1:]
O = O.to_numpy()
output = O @ np.transpose(O)

output = torch.from_numpy(output)
t = CIR(X, Y, Xt, Yt, 0.0001, 2)
torch.set_printoptions(sci_mode=False)
print(t)
# print(torch.subtract(output, t))

# t1 = CIR2(X, Y, Xt, Yt, 0.0001, 2)

# print(np.subtract(output, t1))
# print(output)


tensor([[     0.375201923824,     -0.101333258192,     -0.031706456667,
              0.074056601812,     -0.003391824506,     -0.006406077678,
              0.014818752396,     -0.014912720700,      0.010481833079,
              0.005472523410,     -0.080417297943,      0.002312598928,
             -0.001375736321,     -0.030269470306,     -0.035455826476,
             -0.009230959567,     -0.008247880834,      0.064804866142,
             -0.043434540771,     -0.032812540660,     -0.053186422703,
              0.052297645424,      0.023755323408,     -0.037172064335,
              0.054445387372,     -0.230890239020,     -0.024670452795,
             -0.026377251075,      0.039847729062,      0.025851973874,
             -0.032097759057,     -0.010730613648,     -0.026825819527,
             -0.037574114701,     -0.106742892079,     -0.043429543998,
              0.013944372548,     -0.008602165913,      0.012388197173,
              0.022461043274,     -0.002296669599,     -0.004221

In [ ]:
# t = pd.read_csv('test-fX.csv')
# t = pd.DataFrame(t)
# if t.iloc[:, 0].dtypes != t.iloc[:, 1].dtypes:
#     t = t.iloc[:, 1:]


In [ ]:
# t = CIR(X, Y, Xt, Yt, 0.0001, 2)
# print(torch.subtract(output, t))

# output = torch.from_numpy(output)
# print("Correct output....")
# print(output)

# # n = pd.read_csv('test - noIndex.csv')

# X = pd.read_csv('test - fX.csv')
# Y = pd.read_csv('test - fY.csv')
# Xt = pd.read_csv('test - bX.csv')
# Yt = pd.read_csv('test - fY.csv')